In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tarfile
import wget


tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

In [3]:
print(cv2.__version__)

4.6.0


In [2]:
pretrained_checkpoint = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

if not os.path.exists(pretrained_checkpoint.split('.')[0]):
    download_tar =  'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
    wget.download(download_tar)
    tar = tarfile.open(pretrained_checkpoint)
    tar.extractall()
    tar.close()

## Create label map

In [4]:
label_map = 'label_map.pbtxt'
labels = [{'name':'licence', 'id':1}]

if not os.path.exists(label_map):
    with open(label_map, 'w') as f:
        for label in labels:
            f.write('item { \n')
            f.write(f"\tname: '{label['name']}' \n")
            f.write(f"\tid: {label['id']} \n")
            f.write('}\n')

## Create a TF record

In [4]:
tfrecord = os.path.join('Scripts', 'generate_tfrecord.py')
if not os.path.exists(tfrecord):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {'Scripts'}

In [5]:
print(type(tfrecord))

<class 'str'>


In [ ]:
!python {tfrecord} -x {os.path.join('train')} -l {os.path.join(label_map)} -o {os.path.join('annotations', 'train.record')}
!python {tfrecord} -x {os.path.join('test')} -l {os.path.join(label_map)} -o {os.path.join('annotations', 'test.record')}

In [5]:
new_path = 'model_config/pipeline.config'

In [6]:
configs = config_util.get_configs_from_pipeline_file(new_path)

In [ ]:
configs

In [ ]:
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(new_path, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join('ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = label_map
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join('annotations', 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = label_map
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join('annotations', 'test.record')]

In [ ]:
pipeline_config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(new_path, 'wb') as f:
    f.write(config_text)

## Train the model

In [ ]:
Training_script = r'C:\Users\SHOCKER\tensorflow_projects\models\research\object_detection\model_main_tf2.py'

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(Training_script,
                                                                                             'model_config', new_path)
command

In [ ]:
!{command}

## Load checkpoint

In [6]:
configs = config_util.get_configs_from_pipeline_file(new_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join('model_config', 'ckpt-6')).expect_partial()



In [7]:
@tf.function
def detect_image(image):
    images, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(images, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(label_map)

In [24]:
image_path = r'train\Cars0.png'

image = cv2.imread(image_path)
img = np.array(image)

input_tensor = tf.convert_to_tensor(np.expand_dims(img, 0), dtype=tf.float32)

detections = detect_image(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}


detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = img.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes']+label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw = 5,
    min_score_thresh = 0.5,
    agnostic_mode=False
)

cv2.imshow('object_detec', image_np_with_detections)
cv2.waitKey(0)

-1